In [1]:
%pip install databricks-sdk --upgrade


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install psycopg2-binary


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
def get_all_products(connection):
    """Get all products with profit margins from the syncedcandy table."""
    cursor = connection.cursor()
    cursor.execute("""
        SELECT  * FROM "default".syncedcandy
    """)
    print(cursor.fetchall())

def create_procedure_inline(connection):
    """
    Create the procedure with ROUND fix inline.
    """
    procedure_sql = """
    CREATE OR REPLACE PROCEDURE get_candy_analytics_multiple_results(
        INOUT overall_cursor refcursor,
        INOUT division_cursor refcursor,
        INOUT factory_cursor refcursor
    )
    LANGUAGE plpgsql
    AS $$
    BEGIN
        -- Open cursor for overall statistics
        OPEN overall_cursor FOR
            SELECT 
                COUNT(*) as total_products,
                COUNT(DISTINCT "Division") as total_divisions,
                COUNT(DISTINCT "Factory") as total_factories,
                ROUND(CAST(AVG("Unit Price") AS numeric), 2) as avg_price,
                ROUND(CAST(AVG("Unit Cost") AS numeric), 2) as avg_cost,
                ROUND(CAST(AVG("Unit Price" - "Unit Cost") AS numeric), 2) as avg_profit,
                ROUND(CAST(MIN("Unit Price") AS numeric), 2) as min_price,
                ROUND(CAST(MAX("Unit Price") AS numeric), 2) as max_price,
                ROUND(CAST(SUM("Unit Price" - "Unit Cost") AS numeric), 2) as total_profit
            FROM "default".syncedcandy;
        
        -- Open cursor for division breakdown
        OPEN division_cursor FOR
            SELECT 
                "Division",
                COUNT(*) as product_count,
                ROUND(CAST(AVG("Unit Price") AS numeric), 2) as avg_price,
                ROUND(CAST(AVG("Unit Cost") AS numeric), 2) as avg_cost,
                ROUND(CAST(AVG("Unit Price" - "Unit Cost") AS numeric), 2) as avg_profit,
                ROUND(CAST(SUM("Unit Price" - "Unit Cost") AS numeric), 2) as total_profit,
                ROUND(CAST(MIN("Unit Price") AS numeric), 2) as min_price,
                ROUND(CAST(MAX("Unit Price") AS numeric), 2) as max_price
            FROM "default".syncedcandy
            GROUP BY "Division"
            ORDER BY "Division";
        
        -- Open cursor for factory breakdown
        OPEN factory_cursor FOR
            SELECT 
                "Factory",
                COUNT(*) as product_count,
                ROUND(CAST(AVG("Unit Price") AS numeric), 2) as avg_price,
                ROUND(CAST(AVG("Unit Cost") AS numeric), 2) as avg_cost,
                ROUND(CAST(AVG("Unit Price" - "Unit Cost") AS numeric), 2) as avg_profit,
                ROUND(CAST(SUM("Unit Price" - "Unit Cost") AS numeric), 2) as total_profit,
                ROUND(CAST(MIN("Unit Price") AS numeric), 2) as min_price,
                ROUND(CAST(MAX("Unit Price") AS numeric), 2) as max_price
            FROM "default".syncedcandy
            GROUP BY "Factory"
            ORDER BY "Factory";
            
    EXCEPTION
        WHEN OTHERS THEN
            RAISE NOTICE 'Error in get_candy_analytics_multiple_results: %', SQLERRM;
            RAISE;
    END;
    $$;
    """
    
    try:
        with connection.cursor() as cursor:
            cursor.execute(procedure_sql)
            connection.commit()
            print("✅ Procedure created successfully!")
    except Exception as e:
        print(f"❌ Error: {e}")
        connection.rollback()
        raise

def test_candy_analytics_procedure(connection):
    """
    Test the stored procedure and fetch data from each cursor.
    """
    try:
        with connection.cursor() as cursor:
            # Call the stored procedure with cursor names
            cursor.execute("""CALL get_candy_analytics_multiple_results(%s, %s, %s)""", 
                          ['overall_cur', 'division_cur', 'factory_cur'])
            
            print("✅ Stored procedure executed successfully!")
            print("=" * 60)
            
            # Fetch overall statistics
            print("\n📊 OVERALL STATISTICS:")
            print("-" * 40)
            cursor.execute('FETCH ALL FROM "overall_cur"')
            overall_data = cursor.fetchall()
            
            # Get column names for overall stats
            overall_columns = [desc[0] for desc in cursor.description]
            
            if overall_data:
                overall_row = overall_data[0]
                for i, value in enumerate(overall_row):
                    print(f"{overall_columns[i].replace('_', ' ').title()}: {value}")
            
            # Fetch division breakdown
            print("\n🏭 DIVISION BREAKDOWN:")
            print("-" * 40)
            cursor.execute('FETCH ALL FROM "division_cur"')
            division_data = cursor.fetchall()
            division_columns = [desc[0] for desc in cursor.description]
            
            if division_data:
                print(f"{'Division':<20} {'Count':<8} {'Avg Price':<10} {'Avg Profit':<12} {'Total Profit':<12}")
                print("-" * 70)
                for row in division_data:
                    division_dict = dict(zip(division_columns, row))
                    print(f"{division_dict['Division']:<20} "
                          f"{division_dict['product_count']:<8} "
                          f"${division_dict['avg_price']:<9} "
                          f"${division_dict['avg_profit']:<11} "
                          f"${division_dict['total_profit']:<11}")
            
            # Fetch factory breakdown
            print("\n🏢 FACTORY BREAKDOWN:")
            print("-" * 40)
            cursor.execute('FETCH ALL FROM "factory_cur"')
            factory_data = cursor.fetchall()
            factory_columns = [desc[0] for desc in cursor.description]
            
            if factory_data:
                print(f"{'Factory':<20} {'Count':<8} {'Avg Price':<10} {'Avg Profit':<12} {'Total Profit':<12}")
                print("-" * 70)
                for row in factory_data:
                    factory_dict = dict(zip(factory_columns, row))
                    print(f"{factory_dict['Factory']:<20} "
                          f"{factory_dict['product_count']:<8} "
                          f"${factory_dict['avg_price']:<9} "
                          f"${factory_dict['avg_profit']:<11} "
                          f"${factory_dict['total_profit']:<11}")
            
            # Close the cursors
            cursor.execute('CLOSE "overall_cur"')
            cursor.execute('CLOSE "division_cur"')
            cursor.execute('CLOSE "factory_cur"')
            
            print("\n✅ All data fetched successfully!")
            
            # Return the data for further use if needed
            return {
                'overall': {
                    'columns': overall_columns,
                    'data': overall_data
                },
                'divisions': {
                    'columns': division_columns,
                    'data': division_data
                },
                'factories': {
                    'columns': factory_columns,
                    'data': factory_data
                }
            }
            
    except Exception as e:
        print(f"❌ Error testing stored procedure: {e}")
        raise


# OAuth with User
If you are a database owner, admin, or your Databricks identity has a corresponding Postgres role for the database instance, you can obtain an OAuth token from the UI, the Databricks CLI, or one of the Databricks SDKs (Python SDK or Java SDK) 

See: https://docs.databricks.com/aws/en/oltp/oauth?language=Python+SDK#obtain-an-oauth-token-in-a-user-to-machine-flow 

In [ ]:
# TODO: Update these variables
instance_name = "achille"
database = "databricks_postgres"
port = 5432
user = "anhhoang.chu@databricks.com"

In [ ]:
from databricks.sdk import WorkspaceClient
import uuid

import psycopg
import string
from psycopg_pool import ConnectionPool

w = WorkspaceClient()

instance = w.database.get_database_instance(name=instance_name)
host = instance.read_write_dns
cred = w.database.generate_database_credential(request_id=str(uuid.uuid4()), instance_names=[host])

In [ ]:
# Using Python SDK USING 

import psycopg2
from psycopg2 import sql, pool
import uuid


# Create a connection pool
connection_pool = psycopg2.pool.ThreadedConnectionPool(
    minconn = 1,  # Minimum number of connections in the pool
    maxconn = 10,  # Maximum number of connections in the pool
    user = user,
    password = cred.token,
    host = host,
    port = port,
    sslmode='require',
    database = database
)
if connection_pool:
    print("Connection pool created successfully")

def executeWithPgConnection(execFn):
    connection = None
    try:
        # Get a connection from the pool
        connection = connection_pool.getconn()
        if connection:
            print("Successfully received a connection from the pool")
            execFn(connection)
    finally:
        # Release the connection back to the pool
        if connection:
            connection_pool.putconn(connection)
            print("Connection returned to the pool")

def printVersion(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT version()")
    version = cursor.fetchone()
    print(f"Connected to PostgreSQL database. Version: {version}")

executeWithPgConnection(printVersion)          

Connection pool created successfully
Successfully received a connection from the pool
Connected to PostgreSQL database. Version: ('PostgreSQL 16.8 on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)
Connection returned to the pool


In [42]:
def queryPostgresData(connection):
    cursor = connection.cursor()
    cursor.execute('SELECT COUNT(*) FROM "default"."syncedcandy"')
    count = cursor.fetchone()[0]
    print(f"Connected to database. Found {count} products in syncedcandy table.")

executeWithPgConnection(queryPostgresData)

Successfully received a connection from the pool
Connected to database. Found 15 products in syncedcandy table.
Connection returned to the pool


In [ ]:
executeWithPgConnection(get_all_products)

Successfully received a connection from the pool
[('Chocolate', 'Wonka Bar - Nutty Crunch Surprise', "Lot's O' Nuts", 'CHO-NUT-13000', 3.49, 1.0), ('Chocolate', 'Wonka Bar - Fudge Mallows', "Lot's O' Nuts", 'CHO-FUD-51000', 3.6, 1.2), ('Chocolate', 'Wonka Bar -Scrumdiddlyumptious', "Lot's O' Nuts", 'CHO-SCR-58000', 3.6, 1.1), ('Chocolate', 'Wonka Bar - Milk Chocolate', "Wicked Choccy's", 'CHO-MIL-31000', 3.25, 1.14), ('Chocolate', 'Wonka Bar - Triple Dazzle Caramel', "Wicked Choccy's", 'CHO-TRI-54000', 3.75, 1.3), ('Sugar', 'Laffy Taffy', 'Sugar Shack', 'SUG-LAF-25000', 1.99, 0.75), ('Sugar', 'SweeTARTS', 'Sugar Shack', 'SUG-SWE-91000', 1.5, 0.8), ('Sugar', 'Nerds', 'Sugar Shack', 'SUG-NER-92000', 1.5, 0.8), ('Sugar', 'Fun Dip', 'Sugar Shack', 'SUG-FUN-75000', 1.5, 0.9), ('Other', 'Fizzy Lifting Drinks', 'Sugar Shack', 'OTH-FIZ-56000', 3.75, 1.5), ('Sugar', 'Everlasting Gobstopper', 'Secret Factory', 'SUG-EVE-47000', 10.0, 2.0), ('Sugar', 'Hair Toffee', 'The Other Factory', 'SUG-HAI-55

In [ ]:
executeWithPgConnection(create_procedure_inline)

Successfully received a connection from the pool
✅ Procedure created successfully!
Connection returned to the pool


In [ ]:
# Execute the test
result_data = executeWithPgConnection(test_candy_analytics_procedure)

Successfully received a connection from the pool
✅ Stored procedure executed successfully!

📊 OVERALL STATISTICS:
----------------------------------------
Total Products: 15
Total Divisions: 3
Total Factories: 5
Avg Price: 4.46
Avg Cost: 1.81
Avg Profit: 2.66
Min Price: 1.25
Max Price: 20.00
Total Profit: 39.84

🏭 DIVISION BREAKDOWN:
----------------------------------------
Division             Count    Avg Price  Avg Profit   Total Profit
----------------------------------------------------------------------
Chocolate            5        $3.54      $2.39        $11.95      
Other                4        $7.06      $3.29        $13.15      
Sugar                6        $3.50      $2.46        $14.74      

🏢 FACTORY BREAKDOWN:
----------------------------------------
Factory              Count    Avg Price  Avg Profit   Total Profit
----------------------------------------------------------------------
Lot's O' Nuts        3        $3.56      $2.46        $7.39       
Secret Factory  

# OAuth with Service Principal

To enable secure, automated (machine-to-machine) access to the database instance, you must obtain an OAuth token using a Databricks service principal. This process involves configuring the service principal, generating credentials, and minting OAuth tokens for authentication. 

See: https://docs.databricks.com/aws/en/oltp/oauth?language=Python+SDK#obtain-an-oauth-token-in-a-machine-to-machine-flow 

1. Create a Service Principal
  * Databricks Workspace -> Settings
  * Identity and access
  * Service Principals -> Manage
  * Add service principal

2. Create Client Secrets for the Service Principal
  * Choose the Service Principale -> Secrets
  * Generate Secrets -> Enter the time to live for the secret (OAuth secrets have a maximum lifetime of 730 days (two years))
  * Copy and save the generated Secret and Client ID


In [3]:
import dotenv

dotenv.load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [13]:
# TODO: Update these variables
workspace_url = 'https://e2-demo-field-eng.cloud.databricks.com/'
instance_name = 'achille'
database = "databricks_postgres"
port = 5432

client_id = client_id
client_secret = client_secret
user = "anhhoang.chu@databricks.com"

In [14]:
# Using Python SDK

import psycopg2
from psycopg2 import sql, pool

from databricks.sdk import WorkspaceClient
import uuid

w = WorkspaceClient(host=workspace_url, client_id=client_id, client_secret=client_secret)

instance = w.database.get_database_instance(name=instance_name)
cred = w.database.generate_database_credential(request_id=str(uuid.uuid4()), instance_names=[instance_name])

# Create a connection pool
connection_pool = psycopg2.pool.ThreadedConnectionPool(
    minconn = 1,  # Minimum number of connections in the pool
    maxconn = 10,  # Maximum number of connections in the pool
    user = user,
    password = cred.token,
    host = instance.read_write_dns,
    port = '5432',
    sslmode='require',
    database = database
)
if connection_pool:
    print("Connection pool created successfully")


def executeWithPgConnectionSp(execFn):
    connection = None
    try:
        # Get a connection from the pool
        connection = connection_pool.getconn()
        if connection:
            print("Successfully received a connection from the pool")
            execFn(connection)
    finally:
        # Release the connection back to the pool
        if connection:
            connection_pool.putconn(connection)
            print("Connection returned to the pool")

Connection pool created successfully


In [20]:
executeWithPgConnectionSp(get_all_products)

Successfully received a connection from the pool
[('Chocolate', 'Wonka Bar - Nutty Crunch Surprise', "Lot's O' Nuts", 'CHO-NUT-13000', 3.49, 1.0), ('Chocolate', 'Wonka Bar - Fudge Mallows', "Lot's O' Nuts", 'CHO-FUD-51000', 3.6, 1.2), ('Chocolate', 'Wonka Bar -Scrumdiddlyumptious', "Lot's O' Nuts", 'CHO-SCR-58000', 3.6, 1.1), ('Chocolate', 'Wonka Bar - Milk Chocolate', "Wicked Choccy's", 'CHO-MIL-31000', 3.25, 1.14), ('Chocolate', 'Wonka Bar - Triple Dazzle Caramel', "Wicked Choccy's", 'CHO-TRI-54000', 3.75, 1.3), ('Sugar', 'Laffy Taffy', 'Sugar Shack', 'SUG-LAF-25000', 1.99, 0.75), ('Sugar', 'SweeTARTS', 'Sugar Shack', 'SUG-SWE-91000', 1.5, 0.8), ('Sugar', 'Nerds', 'Sugar Shack', 'SUG-NER-92000', 1.5, 0.8), ('Sugar', 'Fun Dip', 'Sugar Shack', 'SUG-FUN-75000', 1.5, 0.9), ('Other', 'Fizzy Lifting Drinks', 'Sugar Shack', 'OTH-FIZ-56000', 3.75, 1.5), ('Sugar', 'Everlasting Gobstopper', 'Secret Factory', 'SUG-EVE-47000', 10.0, 2.0), ('Sugar', 'Hair Toffee', 'The Other Factory', 'SUG-HAI-55

In [21]:
executeWithPgConnectionSp(create_procedure_inline)

Successfully received a connection from the pool
✅ Procedure created successfully!
Connection returned to the pool


In [22]:
result_data = executeWithPgConnectionSp(test_candy_analytics_procedure)

Successfully received a connection from the pool
✅ Stored procedure executed successfully!

📊 OVERALL STATISTICS:
----------------------------------------
Total Products: 15
Total Divisions: 3
Total Factories: 5
Avg Price: 4.46
Avg Cost: 1.81
Avg Profit: 2.66
Min Price: 1.25
Max Price: 20.00
Total Profit: 39.84

🏭 DIVISION BREAKDOWN:
----------------------------------------
Division             Count    Avg Price  Avg Profit   Total Profit
----------------------------------------------------------------------
Chocolate            5        $3.54      $2.39        $11.95      
Other                4        $7.06      $3.29        $13.15      
Sugar                6        $3.50      $2.46        $14.74      

🏢 FACTORY BREAKDOWN:
----------------------------------------
Factory              Count    Avg Price  Avg Profit   Total Profit
----------------------------------------------------------------------
Lot's O' Nuts        3        $3.56      $2.46        $7.39       
Secret Factory  